In [ ]:
import urllib3
from bs4 import BeautifulSoup
import certifi
import logging

In [ ]:
# ログレベルを DEBUG に変更
formatter = '%(levelname)s : %(asctime)s : %(message)s'
logging.basicConfig(filename='logger.log', level=logging.DEBUG, format=formatter)

# 従来の出力
logging.info('error{}'.format('outputting error'))
logging.info('warning %s %s' % ('was', 'outputted'))
# logging のみの書き方
logging.info('info %s %s', 'test', 'test')

In [ ]:
host = "http://www.panamusica.co.jp"

In [ ]:
def get_page(page):
#     url = host + "/ja/product/genre/104/r/000/" + page + "/"
    url = page
    print(url)
    http = urllib3.PoolManager(
        cert_reqs='CERT_REQUIRED',
        ca_certs=certifi.where())
    r = http.request('GET', url)
    return r

In [ ]:
def get_contents(spec, num):
    return spec.contents[num] if (len(spec) > num) else ''

In [ ]:
def get_string(obj):
    return obj.string if obj is not None else ''

In [ ]:
def output(r):
    soup = BeautifulSoup(r, 'html.parser')
#     print(soup)
    contents = soup.find("ol")

    song_list = contents.find_all("li")
    return_list = []

    for song in song_list:
        
        title = song.h4.string
        spec = song.find("div", class_="spec")
        
        composer = get_contents(spec, 3)
        publisher = get_contents(spec, 5)
        price = get_string(song.strong)
        path = host + song.a.get("href")

    #     print("曲名：" + title)
    #     print("出版社：" + publisher)
    #     print("作編曲者：" + composer)
    #     print("単価（税込）：" + price)
    #     print("リンク：" + path)

        # データをリストに保持
        list = []
        list.append(title)
        list.append(composer)
        list.append(publisher)
        list.append(price)
        list.append(path)
        
        return_list.append('$'.join(list).replace(u'\xa0', u'').replace(u'\u3000', u''))
        print('$'.join(list))
        
    return return_list

In [ ]:
for i in range(10):
    try:
        page = i * 10
        r = get_page(str(page))
        output_list = output(r)
        
        path_w = 'list.txt'
        with open(path_w, mode='a') as f:
            f.write('\n'.join(output_list))
            f.write('\n')
            f.close()
        
        logging.info('Complete!: %d', i)
    except Exception as inst:
        logging.error(inst)